# Algorithmic Fairness, Accountability, and Ethics, Spring 2023

## Mandatory Assignment 1

Please use the following code to prepare the dataset.
 

In [ ]:
# !pip install folktables
# !pip install hvplot

In [ ]:
from folktables.acs import adult_filter
from folktables import ACSDataSource
import numpy as np
import pandas as pd
from tqdm import tqdm
from tqdm import trange
from sklearn.model_selection import train_test_split
import warnings

warnings.filterwarnings("ignore")


data_source = ACSDataSource(survey_year="2018", horizon="1-Year", survey="person")
acs_data = data_source.get_data(states=["CA"], download=True)

feature_names = [
    "AGEP",  # Age
    "CIT",  # Citizenship status
    "COW",  # Class of worker
    "ENG",  # Ability to speak English
    "SCHL",  # Educational attainment
    "MAR",  # Marital status
    "HINS1",  # Insurance through a current or former employer or union
    "HINS2",  # Insurance purchased directly from an insurance company
    "HINS4",  # Medicaid
    "RAC1P",  # Recoded detailed race code
    "SEX",
]

target_name = "PINCP"  # Total person's income


def data_processing(data, features, target_name: str, threshold: float = 35000):
    df = data
    ### Adult Filter (STARTS) (from Foltktables)
    df = df[~df["SEX"].isnull()]
    df = df[~df["RAC1P"].isnull()]
    df = df[df["AGEP"] > 16]
    df = df[df["PINCP"] > 100]
    df = df[df["WKHP"] > 0]
    df = df[df["PWGTP"] >= 1]
    ### Adult Filter (ENDS)
    ### Groups of interest
    sex = df["SEX"].values
    ### Target
    df["target"] = df[target_name] > threshold
    target = df["target"].values
    df = df[
        features + ["target", target_name]
    ]  ##we want to keep df before one_hot encoding to make Bias Analysis
    df_processed = df[features].copy()

    # For some columns, we have binary values of 1 and 2, meaning they either have it or dont
    # There's no reason to dummy encode these, however:
    # It's extremely unintuitive that a value of 2 means that someone doesn't have insurance, therefore we convert to traditional 0=false, 1=true
    # We do this by bit magic: if we minus by 2, then 1 -> -1, but converting this to a bool and back to an int, we get 1 -> -1 -> True -> 1
    # Then we correctly achieve 0=false and 1=true
    cols_where_2_is_falses = [
        "HINS1",
        "HINS2",
        "HINS4",
    ]
    df_processed.loc[:, cols_where_2_is_falses] = (
        (df_processed[["HINS1", "HINS2", "HINS4"]] - 2).astype(bool).astype(int)
    )
    cols = ["CIT", "COW", "SCHL", "MAR", "SEX", "RAC1P"]
    df_processed = pd.get_dummies(
        df_processed,
        prefix=None,
        prefix_sep="_",
        dummy_na=False,
        columns=cols,
        drop_first=True,
    )
    df_processed = pd.get_dummies(
        df_processed,
        prefix=None,
        prefix_sep="_",
        dummy_na=True,
        columns=["ENG"],
        drop_first=True,
    )
    return df_processed, df, target, sex


data, data_original, target, group = data_processing(
    acs_data.sample(10_000, random_state=0), feature_names, target_name
)

X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(
    data, target, group, test_size=0.2, random_state=0
)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

# from sklearn.svm import SVC # Too slow
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt

import shap
import hvplot.pandas
import holoviews as hv
import random
import numpy
hv.notebook_extension('bokeh')
numpy.random.seed(0)
random.seed(0)


In [ ]:
clf1 = make_pipeline(MinMaxScaler(), LogisticRegression(random_state=0))
clf2 = make_pipeline(
    MinMaxScaler(), RandomForestClassifier(max_depth=3, random_state=0)
)
clfs = [clf1, clf2]
for clf in clfs:
    clf.fit(X_train, y_train)

train_group_masks = [group_train == 1, group_train == 2]
test_group_masks = [group_test == 1, group_test == 2]

In [ ]:
print("Baseline accuracy")
for i, clf in enumerate(clfs):
    Accuracy = round(clf.score(X_test, y_test), 3)
    print(clf[1], "\n", Accuracy)

In [ ]:
scores = [
    [clf.score(X_test[mask], y_test[mask]) for mask in test_group_masks] for clf in clfs
]
pd.DataFrame(
    scores, columns=["Group 1", "Group 2"], index=["WhiteBox", "BlackBox"]
).rename_axis("Baseline accuracy [%]").round(2)

In [ ]:
df_metrics = pd.DataFrame([], index=["WhiteBox", "BlackBox"])
df_boxes = ["WhiteBox", "BlackBox"]

positives = [
    [clf.predict(X_test[mask]).mean() for mask in test_group_masks] for clf in clfs
]
df_metrics.loc["WhiteBox", "SP_1"] = positives[0][0]
df_metrics.loc["WhiteBox", "SP_2"] = positives[0][1]
df_metrics.loc["BlackBox", "SP_1"] = positives[1][0]
df_metrics.loc["BlackBox", "SP_2"] = positives[1][1]
parity_df = (
    pd.DataFrame(
        positives, columns=["Group 1", "Group 2"], index=["WhiteBox", "BlackBox"]
    )
    .rename_axis("Baseline positive [%]")
    .round(2)
)
parity_df

In [ ]:
plot_df = (
    parity_df.stack()
    .rename_axis(["box", "category"])
    .rename("prob")
    .reset_index()
    .assign(color=["#4477AA", "#EE6677"] * 2)
    .replace({"Group 1": "Pr(Selection | Male)", "Group 2": "Pr(Selection | Female)"})
)
plots = plot_df.hvplot.bar(
    x="category", width=600, groupby="box", color="color", invert=True, ylim=(0, 1)
)
plot_df

In [ ]:
parity_plots = (
    plots["WhiteBox"].opts(title="Statistical Parity (Whitebox)")
    + plots["BlackBox"].opts(
        width=400, yaxis=None, title="Statistical Parity (Blackbox)"
    )
).opts(shared_axes=False)

In [ ]:
def get_group_df(group, group_no, clf):
    prob = pd.Series(clf.predict_proba(X_train[group])[:, 1], name="prob")
    T = pd.Series(y_train[group], name="T")
    G = pd.Series([bool(group_no)] * len(T), name="G")
    return pd.concat([prob, G, T], axis=1)


for i, box in enumerate(df_boxes):
    df = pd.concat(
        [
            get_group_df(group, group_no, clfs[i])
            for group_no, group in enumerate(train_group_masks)
        ],
    )
    df["S"] = df["prob"] * 100 > 50
    # G is group
    # T is target
    # S is prediction/selection
    N = lambda string: len(df.query(string))
    df_metrics.loc[box, "E_odd_t0_g0"] = N("~T and S and ~G") / N("~G and ~T")
    df_metrics.loc[box, "E_odd_t0_g1"] = N("~T and S and G") / N("G and ~T")
    df_metrics.loc[box, "E_odd_t1_g0"] = N("T and S and ~G") / N("~G and T")
    df_metrics.loc[box, "E_odd_t1_g1"] = N("T and S and G") / N("G and T")
    # Equalized outcome 0 is precisio
    df_metrics.loc[box, "E_out_s0_g0"] = N("~G and T and ~S") / N("~G and ~S")
    df_metrics.loc[box, "E_out_s0_g1"] = N("G and T and ~S") / N("G and ~S")
    df_metrics.loc[box, "E_out_s1_g0"] = N("~G and T and S") / N("~G and S")
    df_metrics.loc[box, "E_out_s1_g1"] = N("G and T and S") / N("G and S")

In [ ]:
name_map = {
    "E_odd_t0_g0": "Pr(Selection | low income, Male)",
    "E_odd_t0_g1": "Pr(Selection | low income, Female)",
    "E_odd_t1_g0": "Pr(Selection | high income, Male)",
    "E_odd_t1_g1": "Pr(Selection | high income, Female)",
    "E_out_s0_g0": "Pr(High income | not selected, Male)",
    "E_out_s0_g1": "Pr(High income | not selected, Female)",
    "E_out_s1_g0": "Pr(High income | selected, Male)",
    "E_out_s1_g1": "Pr(High income | selected, Female)",
}
color_map = {
    "E_odd_t0_g0": "#4477AA",
    "E_odd_t0_g1": "#EE6677",
    "E_odd_t1_g0": "#4477AA",
    "E_odd_t1_g1": "#EE6677",
    "E_out_s0_g0": "#4477AA",
    "E_out_s0_g1": "#EE6677",
    "E_out_s1_g0": "#4477AA",
    "E_out_s1_g1": "#EE6677",
}

plot_kwargs = dict(
    x="category",
    invert=True,
    ylim=(0, 1),
    color="color",
)


def get_eq_odds_plot(model_type: str, df_metrics=df_metrics):
    df = (
        df_metrics[[col for col in df_metrics if "odd" in col]]
        .loc[model_type]
        .reset_index()
    )
    plot_df = df.assign(
        category=df["index"].map(name_map),
        color=df["index"].map(color_map),
    )
    return plot_df.hvplot.bar(
        title=f"Equalized Odds ({model_type})", y=model_type, **plot_kwargs
    )


def get_eq_out_plot(model_type: str, df_metrics=df_metrics):
    df = (
        df_metrics[[col for col in df_metrics if "out" in col]]
        .loc[model_type]
        .reset_index()
    )
    plot_df = df.assign(
        category=df["index"].map(name_map),
        color=df["index"].map(color_map),
    )
    return plot_df.hvplot.bar(
        title=f"Equalized Outcomes ({model_type})", y=model_type, **plot_kwargs
    )


plots = (
    (
        parity_plots[0].opts(width=450)
        + parity_plots[1].opts(width=250)
        + get_eq_odds_plot("WhiteBox").opts(width=450)
        + get_eq_odds_plot("BlackBox").opts(yaxis=None, width=250)
        + get_eq_out_plot("WhiteBox").opts(width=450)
        + get_eq_out_plot("BlackBox").opts(yaxis=None, width=250)
    )
    .cols(2)
    .opts(shared_axes=False)
    .opts(hv.opts.Bars(ylabel="", xlabel="", xaxis=None, height=150))
)
plots[-1].opts(xaxis=True)
plots[-2].opts(xaxis=True)
plots

In [ ]:
def ROC_curve(clf):
    def get_group_df(group, group_no):
        prob = pd.Series(clf.predict_proba(X_train[group])[:, 1], name="prob")
        T = pd.Series(y_train[group], name="T")
        G = pd.Series([bool(group_no)] * len(T), name="G")
        return pd.concat([prob, G, T], axis=1)

    df = pd.concat(
        [
            get_group_df(group, group_no)
            for group_no, group in enumerate(train_group_masks)
        ],
    )

    ROC_g = []
    ROC_G = []
    for threshold in range(5, 100, 5):
        df["S"] = df["prob"] * 100 > threshold
        # TP_G = Count(S, G, T) / Count(G, T)
        # FP_G = Count(S, G, ~T) / Count(G, ~T)
        TP_g = len(df.query("S and ~G and T")) / len(df.query("~G and T"))
        FP_g = len(df.query("S and ~G and ~T")) / len(df.query("~G and ~T"))
        ROC_g.append([TP_g, FP_g, threshold])
        TP_G = len(df.query("S and G and T")) / len(df.query("G and T"))
        FP_G = len(df.query("S and G and ~T")) / len(df.query("G and ~T"))
        ROC_G.append([TP_G, FP_G, threshold])

        if threshold == 50:
            g_mid = [[TP_g, FP_g, 50]]
            G_mid = [[TP_G, FP_G, 50]]

    kwargs = dict(
        y="0",
        x="1",
        hover_cols=["2"],
        xlabel="False Positive Rate = Pr(S=1 | G=g, T=0)",
        ylabel="True Positive Rate = Pr(S=1 | G=g, T=1)",
    )
    ROC_plot = pd.DataFrame(ROC_g).hvplot(**kwargs).relabel("G=1") * pd.DataFrame(
        ROC_G
    ).hvplot(**kwargs).relabel("G=2")
    mids = pd.DataFrame(g_mid).hvplot.scatter(**kwargs) * pd.DataFrame(
        G_mid
    ).hvplot.scatter(**kwargs)
    return ROC_plot * mids


plots = ROC_curve(clfs[0]).opts(title=str(clfs[0])) + ROC_curve(clfs[1]).opts(
    title=str(clfs[1])
)
plots.cols(1)

# Creating statistical parity

In [ ]:
def statistical_parity(pred: np.array, lowest_pred) -> np.array:
    ratio_delta = pred.mean() - lowest_pred
    n_remove = int(len(pred) * ratio_delta)
    remove_idx = np.random.choice(np.where(pred == 1)[0], size=n_remove, replace=False)
    pred[remove_idx] = 0
    return pred


predictions = []
scores = []
for clf in clfs:
    preds = [clf.predict(X_test[mask]) for mask in test_group_masks]
    lowest_pred = min(pred.mean() for pred in preds)
    preds = [statistical_parity(pred, lowest_pred) for pred in preds]
    predictions.append(preds)
    scores.append([pred.mean() for pred in preds])

new_metrics = (
    pd.DataFrame(scores, columns=["SP_1", "SP_2"], index=["WhiteBox", "BlackBox"])
    .rename_axis("Parity positive [%]")
    .round(2)
)
new_metrics

In [ ]:
def get_new_group_df(group, group_no, clf):
    S = pd.Series(predictions[clf][group_no], name="S")
    T = pd.Series(y_test[group], name="T")
    G = pd.Series([bool(group_no)] * len(T), name="G")
    return pd.concat([S, G, T], axis=1)


for i, box in enumerate(df_boxes):
    df = pd.concat(
        [
            get_new_group_df(group, group_no, i)
            for group_no, group in enumerate(test_group_masks)
        ],
    )
    # G is group
    # T is target
    # S is prediction/selection
    N = lambda string: len(df.query(string))
    new_metrics.loc[box, "E_odd_t0_g0"] = N("~T and S and ~G") / N("~G and ~T")
    new_metrics.loc[box, "E_odd_t0_g1"] = N("~T and S and G") / N("G and ~T")
    new_metrics.loc[box, "E_odd_t1_g0"] = N("T and S and ~G") / N("~G and T")
    new_metrics.loc[box, "E_odd_t1_g1"] = N("T and S and G") / N("G and T")
    # Equalized outcome 0 is precision
    new_metrics.loc[box, "E_out_s0_g0"] = N("~G and T and ~S") / N("~G and ~S")
    new_metrics.loc[box, "E_out_s0_g1"] = N("G and T and ~S") / N("G and ~S")
    new_metrics.loc[box, "E_out_s1_g0"] = N("~G and T and S") / N("~G and S")
    new_metrics.loc[box, "E_out_s1_g1"] = N("G and T and S") / N("G and S")

In [ ]:
plot_df = (
    parity_df.stack()
    .rename_axis(["box", "category"])
    .rename("prob")
    .reset_index()
    .assign(color=["#4477AA", "#EE6677"] * 2)
    .replace({"Group 1": "Pr(Selection | Male)", "Group 2": "Pr(Selection | Female)"})
)
plots = plot_df.hvplot.bar(
    x="category", width=600, groupby="box", color="color", invert=True, ylim=(0, 1)
)
plot_df

parity_plots = (
    plots["WhiteBox"].opts(title="Statistical Parity (Whitebox)")
    + plots["BlackBox"].opts(
        width=400, yaxis=None, title="Statistical Parity (Blackbox)"
    )
).opts(shared_axes=False)

plots = (
    (
        parity_plots[0].opts(width=450)
        + parity_plots[1].opts(width=250)
        + get_eq_odds_plot("WhiteBox", df_metrics=new_metrics).opts(width=450)
        + get_eq_odds_plot("BlackBox", df_metrics=new_metrics).opts(
            yaxis=None, width=250
        )
        + get_eq_out_plot("WhiteBox", df_metrics=new_metrics).opts(width=450)
        + get_eq_out_plot("BlackBox", df_metrics=new_metrics).opts(
            yaxis=None, width=250
        )
    )
    .cols(2)
    .opts(shared_axes=False)
    .opts(hv.opts.Bars(ylabel="", xlabel="", xaxis=None, height=150))
)
plots[-1].opts(xaxis=True)
plots[-2].opts(xaxis=True)
plots

In [ ]:
new_metrics = new_metrics.rename_axis("")
new_metrics

In [ ]:
# Difference for each metric before doing statistical parity
between_group_difference_before = pd.DataFrame()
between_group_difference_before["E_odd_false_positive"] = (
    df_metrics["E_odd_t0_g0"] - df_metrics["E_odd_t0_g1"]
)
between_group_difference_before["E_odd_true_positive"] = (
    df_metrics["E_odd_t1_g0"] - df_metrics["E_odd_t1_g1"]
)
between_group_difference_before["E_out_false_positive"] = (
    df_metrics["E_out_s0_g0"] - df_metrics["E_out_s0_g1"]
)
between_group_difference_before["E_out_true_positive"] = (
    df_metrics["E_out_s1_g0"] - df_metrics["E_out_s1_g1"]
)
between_group_difference_before 

In [ ]:
# Difference for each metric after doing statistical parity
between_group_difference_after = pd.DataFrame()
between_group_difference_after["E_odd_false_positive"] = (
    new_metrics["E_odd_t0_g0"] - new_metrics["E_odd_t0_g1"]
)
between_group_difference_after["E_odd_true_positive"] = (
    new_metrics["E_odd_t1_g0"] - new_metrics["E_odd_t1_g1"]
)
between_group_difference_after["E_out_false_positive"] = (
    new_metrics["E_out_s0_g0"] - new_metrics["E_out_s0_g1"]
)
between_group_difference_after["E_out_true_positive"] = (
    new_metrics["E_out_s1_g0"] - new_metrics["E_out_s1_g1"]
)
between_group_difference_after

In [ ]:
baseline_inequality = abs(between_group_difference_before) 
after_statistical_parity_inequality = abs(between_group_difference_after)
decrease_in_inequality = baseline_inequality - after_statistical_parity_inequality
display(decrease_in_inequality.round(2))
# Positive value means closer to equality.
# We can see the white box model get closer to equalized odds but much further away from equalized outcomes
# The black box model doesn't change as much since it was much closer to statistical parity already. It gets further away from equalized odds but closer to equalized outcomes.

## Task 2
1. Explain the trained model of your white-box approach (logistic regression
or the decision tree). In particular, discuss which features in the model
are deemed most relevant. Reflect on the interpretation. Does it fit your
intuition about the prediction task?



In [ ]:
lr_clf = make_pipeline(MinMaxScaler(), LogisticRegression(random_state=0))
coefs = []
coef = lr_clf.fit(X_train, y_train)[1].coef_
number_of_samples = 100
for i in trange(number_of_samples):
    X_train_sample = X_train.assign(y_train=y_train).sample(frac=1.0, replace=True)
    y_train_sample = X_train_sample["y_train"]
    X_train_sample = X_train[X_train.columns]
    coefs.append(lr_clf.fit(X_train_sample, y_train_sample)[1].coef_)
coefs_ = [list(coef[0]) for coef in coefs]
coef_df = pd.DataFrame(coefs_, columns=X_train.columns)
coef_stds = coef_df.std()
coef_stderr = coef_stds / number_of_samples
coef_stderr

ab_coef = np.abs(coef).reshape(
    54,
)
feature_importance = ab_coef / coef_stderr
np.argsort(feature_importance)
feature_importance_sorted = feature_importance.sort_values(ascending=False)
feature_importance_sorted.head(5).sort_values().hvplot.bar(
    invert=True, width=300, height=150, title="Top 5 Feature Importance"
)

Task 2 part 1. Feature importance shows that sex , Age and university schooling have large predictive power for income. This makes sense as the older a person is the further on in their career they will be. Gender pay discrimination in favor of men is also a thing that would therefore improve the predictive power of the model. University level schooling gives access to higher paying jobs and thus it makes sense that this feature has high predictive power for income. 

In [ ]:
X = X_train.copy()
sex_col = X_train.columns.to_list().index('SEX_2')
baseline_sex = X_test.iloc[0, sex_col]
baseline_pred = clf1.predict(X.iloc[0:1])[0]
X.iloc[0, sex_col] = ~baseline_sex
counterfactual_pred = clf1.predict(X.iloc[0:1])[0]
print(f'{baseline_pred = }\n{counterfactual_pred = }')
assert baseline_pred != counterfactual_pred

Sex_2 is whether the person is female i.e 1 they are female.
This example shows that when you change the sex, the classification can change

In [ ]:
plots = []
explainers = [
    shap.Explainer,
    shap.TreeExplainer,
]
for clf, explainer in zip(clfs, explainers):
    n_samples_shown = 1000
    clf.fit(X_train, y_train)
    # Shap doesn't work with pipeline classifiers
    transformer, model = [obj for name, obj in clf.steps]
    data = pd.DataFrame(
        transformer.transform(X_train), columns=X_train.columns, index=X_train.index
    )
    sample = data.sample(n_samples_shown)

    shap_values = explainer(model, data).shap_values(sample)
    if isinstance(shap_values, list):
        shap_values = shap_values[1]

    plt.figure()
    plt.title(str(model))
    shap.summary_plot(shap_values, sample, feature_names=X_train.columns)